# 📘 Multi-Task Gaussian Process with Uncertainty Quantification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/04_MTGP_Prediction.ipynb)

This notebook demonstrates **Multi-Task Gaussian Processes (MTGP)** for predicting multiple AD outputs (SCOD, VFA, Biogas) simultaneously with **uncertainty estimation**.

---

## 📚 References

- **MTGP for AD Processes**: [Dekhici et al. (2025) - LAPSE](https://psecommunity.org/LAPSE:2025.0155)

## 🔬 Multi-Task GP Background

### Gaussian Process Regression

A GP defines a distribution over functions:

$$f(x) \sim \mathcal{GP}(m(x), k(x, x'))$$

Where:
- $m(x)$ = mean function
- $k(x, x')$ = covariance (kernel) function

### Multi-Task Learning with Linear Model of Coregionalization (LMC)

For $T$ tasks, we use latent functions $\{u_q(x)\}_{q=1}^{Q}$:

$$f_t(x) = \sum_{q=1}^{Q} a_{t,q} \cdot u_q(x)$$

This allows **information sharing** between correlated tasks (e.g., VFA and Biogas).

### Predictive Distribution

$$p(f_* | X_*, X, Y) = \mathcal{N}(\mu_*, \Sigma_*)$$

Where:
- $\mu_*$ = predicted mean
- $\Sigma_*$ = uncertainty (variance)

### Advantages

1. **Uncertainty Quantification**: Provides confidence intervals
2. **Multi-output**: Predicts correlated outputs jointly
3. **Data Efficient**: Works well with limited data
4. **Non-parametric**: Flexible function approximation

## 1️⃣ Setup (Google Colab)

In [ ]:
# Install OpenAD-lib with ML support (uncomment for Colab)
# !pip install "git+https://github.com/benmola/OpenAD-lib.git#egg=openad_lib[ml]"\n
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openad_lib.models.ml import MultitaskGP

print("✅ All imports successful!")

## 2️⃣ Load Multi-Output Data

We have:
- **Inputs**: time, D (dilution rate), SCODin, OLR, pH
- **Outputs**: SCODout, VFAout, Biogas

In [ ]:
# Load data
if IN_COLAB:
    !wget -q https://raw.githubusercontent.com/benmola/OpenAD-lib/main/src/openad_lib/data/sample_ad_process_data.csv
    data_path = 'sample_ad_process_data.csv'
else:
    base_path = os.path.dirname(os.getcwd())
    data_path = os.path.join(base_path, 'src', 'openad_lib', 'data', 'sample_ad_process_data.csv')

data = pd.read_csv(data_path)
print(f"📊 Loaded {len(data)} samples")
data.head()

In [ ]:
# Define columns explicitly
input_cols = ['time', 'D', 'SCODin', 'OLR', 'pH']
output_cols = ['SCODout', 'VFAout', 'Biogas']

X = data[input_cols].values
Y = data[output_cols].values

print(f"Input shape: {X.shape}")
print(f"Output shape: {Y.shape}")

In [ ]:
# Alternating train/test split (matching reference methodology)
train_indices = np.arange(1, len(X), 2)
test_indices = np.arange(0, len(X), 2)

X_train, X_test = X[train_indices], X[test_indices]
Y_train, Y_test = Y[train_indices], Y[test_indices]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

## 3️⃣ Build and Train MTGP Model

The `MultitaskGP` class uses:
- **GPyTorch** for GP implementation
- **LMC** (Linear Model of Coregionalization) for multi-task learning
- **Variational inference** for scalability

In [ ]:
# Initialize MTGP
mtgp = MultitaskGP(
    num_tasks=3,
    num_latents=3,
    n_inducing=60,
    learning_rate=0.1,
    log_transform=True  # Handle positive outputs
)

print("🚀 Training MTGP model (500 epochs)...")
mtgp.fit(X_train, Y_train, epochs=500, verbose=True)

## 4️⃣ Prediction with Uncertainty

The GP provides:
- **Mean prediction**: $\mu_*$
- **95% Confidence interval**: $[\mu_* - 2\sigma_*, \mu_* + 2\sigma_*]$

In [ ]:
# Predict with uncertainty
mean, lower, upper = mtgp.predict(X_test, return_std=True)

print("📊 Prediction shapes:")
print(f"   Mean: {mean.shape}")
print(f"   Lower (2.5%): {lower.shape}")
print(f"   Upper (97.5%): {upper.shape}")

In [ ]:
# Evaluate
metrics = mtgp.evaluate(X_test, Y_test, task_names=output_cols)

print("📈 MTGP Test Metrics:")
for task, vals in metrics.items():
    print(f"   {task}: RMSE={vals['rmse']:.4f}, R²={vals['r2']:.4f}")

In [ ]:
# Plot results with uncertainty
plt.style.use('bmh')
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

for i, task_name in enumerate(output_cols):
    ax = axes[i]
    
    # Training data
    ax.plot(X_train[:, 0], Y_train[:, i], 'bo', 
            markersize=5, alpha=0.4, label='Train Data')
    
    # Test data
    ax.plot(X_test[:, 0], Y_test[:, i], 'ro', 
            markersize=6, alpha=0.7, label='Test Data')
    
    # Prediction mean
    ax.plot(X_test[:, 0], mean[:, i], 'k-', 
            linewidth=2, label='MTGP Mean')
    
    # Confidence interval
    ax.fill_between(X_test[:, 0], lower[:, i], upper[:, i],
                    color='gray', alpha=0.3, label='95% Confidence')
    
    ax.set_ylabel(task_name, fontsize=14, fontweight='bold')
    ax.set_title(f'{task_name} Prediction with Uncertainty', fontsize=16, pad=10)
    ax.legend(fontsize=10, loc='upper right')
    ax.grid(True, linestyle='--', alpha=0.7)

axes[-1].set_xlabel('Time', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 📝 Summary

This notebook demonstrated:

1. **MTGP Architecture** using LMC for multi-task learning
2. **Joint Training** of correlated outputs (SCOD, VFA, Biogas)
3. **Uncertainty Quantification** with 95% confidence intervals
4. **Visualization** of predictions with uncertainty bands

### Key Advantages over LSTM

| Feature | MTGP | LSTM |
|---------|------|------|
| Uncertainty | ✅ Built-in | ❌ Requires ensembles |
| Multi-output | ✅ Correlated | ❌ Independent |
| Small data | ✅ Works well | ❌ Needs more data |

### Next Steps

- Compare with [LSTM model](03_LSTM_Prediction.ipynb)
- Apply to [MPC Control](05_MPC_Control.ipynb)
- Explore [ADM1 mechanistic model](01_ADM1_Tutorial.ipynb)